In [1]:
from datasets import load_dataset 

# load a custom dataset from local/remote files or folders using the ImageFolder feature

dataset = load_dataset("imagefolder", data_dir="/kaggle/input/thermography-images-of-diabetic-foot/ThermoDataBase/val", split="train")
dataset

Resolving data files:   0%|          | 0/422 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 422
})

In [2]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[1]

'DM Group'

In [3]:
import glob

paths = glob.glob("/kaggle/input/thermography-images-of-diabetic-foot/ThermoDataBase/val/*/*")
len(paths)

422

In [4]:
import numpy as np

In [5]:
y_true = []

for p in paths:
    f = p.split('/')[6]
    y_true.append(label2id[f])

y_true = np.array(y_true)
y_true.shape

(422,)

In [7]:
from transformers import pipeline

pipe = pipeline("image-classification", "Nitish2801/vit-base-patch16-224-in21k-finetuned-footulcer")

config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

In [8]:
results = pipe(paths, num_workers=2, batch_size=8, top_k=1)
results[:5]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[[{'label': 'DM Group', 'score': 0.9710755348205566}],
 [{'label': 'Control Group', 'score': 0.7378173470497131}],
 [{'label': 'DM Group', 'score': 0.971368134021759}],
 [{'label': 'DM Group', 'score': 0.935728132724762}],
 [{'label': 'DM Group', 'score': 0.9735349416732788}]]

In [9]:
y_pred = []

for l in results:
    f = l[0]['label']
    y_pred.append(label2id[f])

y_pred = np.array(y_pred)
y_pred.shape

(422,)

In [10]:
label2id

{'Control Group': 0, 'DM Group': 1}

In [11]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=["Control Group", "DM Group"]))

               precision    recall  f1-score   support

Control Group       0.93      0.80      0.86       170
     DM Group       0.88      0.96      0.92       252

     accuracy                           0.90       422
    macro avg       0.90      0.88      0.89       422
 weighted avg       0.90      0.90      0.89       422

